In [1]:
from typing import List
from pydantic import BaseModel, Field

from langchain_google_vertexai import ChatVertexAI

from article.sources import sources
from article.utils import flatten_openapi

import mistune
from IPython.display import Markdown

In [2]:
class Section(BaseModel):
    name: str = Field(title='Name/title of the section')
    points: List[str] = Field(title='List of no more than 3 main points for this section')
    summary: str = Field(title="Summary of the section in 20-50 words")
    experts: List[str] = Field(title="List of 3-5 experts roles/descrptions who can help with article by providing feedback")

    def __str__(self):
        return f'## {self.name}\n{self.summary}\n\n'+','.join(self.points)

class Outline(BaseModel):
    title: str = Field(title="Title of the article")
    summary: str = Field(title="Summary of the article in 20-50 words")
    sections: List[Section]

    def __str__(self):
        return f'# {self.title}\n{self.summary}\n\n' + '\n'.join([str(x) for x in self.sections])

In [3]:
articles = sources()
style_articles = [x[1] for x in articles if x[0].startswith('data/oreilly')]
articles = [x[1] for x in articles]

### Get style guid

Replicate article style

In [32]:
prompt = [
    ('system', 'You are analyzing articles. You sources are' + '\n'.join(style_articles)),
    ('user', 'Give concise descrition of audeince and style of those articles. Write it as recommendation for another authors to follow when writting simmilar articles. Create sections: Audience, Style, Recommendations. Use Heading 2 for section')
]

In [7]:
model = ChatVertexAI(model='gemini-1.5-flash-002')

In [8]:
response = model.invoke(prompt)

In [9]:
style_guide = response.content

In [10]:
Markdown(style_guide)

## Audience

The target audience for these articles is a broad range of individuals and teams involved in building applications with Large Language Models (LLMs).  This includes practitioners with varying levels of machine learning expertise, from experienced engineers to hobbyist developers working on weekend projects.  The articles aim to be accessible to those without deep ML expertise while providing valuable insights for experienced professionals.


## Style

The writing style is conversational, practical, and opinionated. The authors share their personal experiences and lessons learned, making the content engaging and relatable.  They avoid overly technical jargon, preferring clear explanations and real-world examples.  The tone is collaborative and community-oriented, inviting reader engagement and discussion.  A blend of formal explanations with informal, almost conversational language is utilized.  The authors also use humor and relatable analogies to make complex topics more approachable.


## Recommendations

For authors writing similar articles about LLMs and their applications, consider the following:

* **Target a diverse audience:**  Cater to a spectrum of technical skills, ensuring the content is accessible to both beginners and experts.
* **Adopt a practical and conversational tone:**  Emphasize real-world applications, lessons learned, and relatable examples. Avoid excessive technical detail unless absolutely necessary.
* **Incorporate personal experiences and anecdotes:**  Sharing personal experiences and lessons learned makes the content more engaging and relatable.
* **Embrace a collaborative and community-oriented approach:**  Encourage reader feedback, interaction, and discussion.
* **Balance formality and informality:**  While maintaining professionalism, use a conversational tone to increase reader engagement.
* **Use visuals and examples effectively:**  Include charts, graphs, and code snippets to illustrate complex concepts and enhance understanding.
* **Structure content logically:**  Break down information into digestible sections with clear headings and subheadings.
* **Clearly define target audience and skill level:**  Ensure the chosen style aligns with the desired audience.
* **Proofread carefully for clarity and consistency:**  Maintain high editorial standards to ensure credibility and readability.


In [11]:
markdown = mistune.create_markdown(renderer=None)
sg = markdown(style_guide)
style_sections = [x['children'][0]['raw'] for x in sg if x['type']=='heading']
assert style_sections == ['Audience', 'Style', 'Recommendations']

### Create outline

In [22]:
title = 'My experience with building GenAI Appls in legacy environments'
summary = 'Building new applications in existing environment is always a challenge. We see a lot of GenAI applications in startups or smaller companies. How large companies with legacy IT architectures handle challenge with building GenAI apps.' 

prompt = [
    ('system', 'You are preparing to write new article. Follow the instruction in the schema. You can chance title and/or provided summary. You sources are' + '\n'.join(articles)),
    ('user', f'Prepare outline and plan for new article. Article title {title}\Description: {summary}\nUse style guide: {style_guide}')
]

In [13]:
o_model = ChatVertexAI(model='gemini-1.5-pro-002').with_structured_output(schema=flatten_openapi(Outline.schema()), method='json_mode')

In [14]:
response = o_model.invoke(prompt)

In [15]:
oresponse = Outline.parse_obj(response)

In [19]:
Markdown(str(oresponse))

# My Experience with Building GenAI Applications in Legacy Environments
This article explores the challenges and strategies for building GenAI applications in legacy IT environments. It also features real-world case studies of successful GenAI integrations.

## Challenges of GenAI in Legacy Environments
Legacy IT environments often present significant challenges when integrating GenAI applications. These include technical debt, data silos, security concerns, integration complexities, and talent gaps.

Technical Debt and Compatibility Issues,Data Silos and Accessibility,Security and Compliance Concerns,Integration with Existing Workflows,Talent Acquisition and Upskilling
## Strategies for Successful GenAI Integration
To successfully integrate GenAI into legacy environments, companies must adopt strategies for modernization, data management, security, scalability, and innovation.

Modernization and Cloud Migration,Data Integration and Management,Addressing Security and Compliance,Building Scalable and Reliable Systems,Fostering a Culture of Innovation
## Real-World Case Studies
Real-world case studies demonstrate how companies are effectively integrating GenAI into their legacy environments.

[Company A] - Transforming Customer Service with GenAI,[Company B] - Automating Legacy Processes,[Company C] - Driving Innovation with AI-Powered Insights

In [34]:
prompt2 = prompt
prompt2.append(('ai', str(oresponse)))

In [35]:
prompt2.append(('user', 'Change first section to more focus on difference betwen building app in new environment to doing the same thing in legacy (existing processes, API not ready for such apps, documents not avialable etc'))

In [36]:
response2 = o_model.invoke(prompt2)

In [38]:
response2

{'sections': [{'experts': ['Software engineers',
    'Data scientists',
    'IT architects'],
   'name': 'Challenges of Building GenAI Apps in Legacy Environments',
   'points': ['Existing processes and APIs may not be compatible with GenAI applications.',
    'Limited documentation can hinder integration efforts.',
    'Data silos and security concerns pose additional challenges.',
    'Talent gaps in AI/ML expertise can slow down development.'],
   'summary': 'Building GenAI applications in legacy environments differs significantly from new environments. Existing processes, APIs, and documentation may not be ready for GenAI integration, requiring extra effort.'},
  {'experts': ['Cloud architects', 'Data engineers', 'Security specialists'],
   'name': 'Strategies for Successful GenAI Integration',
   'points': ['Modernization and cloud migration are crucial for compatibility.',
    'Data integration and management break down data silos.',
    'Security and compliance must be addressed